# Installation

Note: If you are on Binder, you don't need to execute the following command.

In [ ]:
!pip install nexusforge

# Configuration

In [ ]:
config = dict()

## Model

Choose one of the following models:

### DemoModel

In [ ]:
config['Model'] = {
    "name": "DemoModel",
    "origin": "directory",
    "source": "../../../tests/data/demo-model/",
}

### Neuroshapes

SHACL-based model: https://github.com/INCF/neuroshapes.

#### from a directory

In [ ]:
neuroshapes_path = "../../models/neuroshapes"

In [ ]:
! rm -Rf $neuroshapes_path

In [ ]:
! git clone https://github.com/INCF/neuroshapes.git $neuroshapes_path

In [ ]:
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/dataset $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/activity $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/entity $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/ontology $neuroshapes_path/shapes/neurosciencegraph/commons/
! cp -R $neuroshapes_path/shapes/neurosciencegraph/datashapes/core/person $neuroshapes_path/shapes/neurosciencegraph/commons/

In [ ]:
config['Model'] = {
    "name": "RdfModel",
    "origin": "directory",
    "source": f"{neuroshapes_path}/shapes/neurosciencegraph/commons/",
    "context": {
        "iri": "../../models/neuroshapes_context.json",
    },
}

#### from an URL

This feature is not yet implemented.

#### from a store

##### BlueBrainNexus

In [ ]:
config['Model'] = {
    "name": "RdfModel",
    "origin": "store",
    "source": "BlueBrainNexus",
    # bucket: <when 'origin' is 'store', a Store bucket>
    "context": {
        "iri": "https://bbp.neuroshapes.org",
        "bucket": "neurosciencegraph/datamodels",
    },
}

## Store

Choose one of the following stores:

### DemoStore

In [ ]:
config["Store"] = {
    "name": "DemoStore",
    "versioned_id_template": "{x.id}?_version={x._store_metadata.version}",
}

### BlueBrainNexus

The [Nexus web application](https://sandbox.bluebrainnexus.io/web) can be used to login and get a token.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/copy-token.png)


In [ ]:
import getpass

token = getpass.getpass()

Create a project:

In [ ]:
import nexussdk as nexus

org ="tutorialnexus"
project ="myProject"
description = "My project description"
endpoint = "https://sandbox.bluebrainnexus.io/v1"

nexus.config.set_environment(endpoint)
nexus.config.set_token(token)

nexus.projects.create(org_label=org,project_label=project, description=description)

In [ ]:
config["Store"] = {
    "name": "BlueBrainNexus",
    "endpoint": endpoint,
    "searchendpoints":{
        "sparql":{
            "endpoint":"https://bluebrain.github.io/nexus/vocabulary/defaultSparqlIndex"
        }
    },
    "bucket": f"{org}/{project}",
    "token": token,
    "versioned_id_template": "{x.id}?rev={x._store_metadata._rev}",
    "file_resource_mapping": "../../configurations/nexus-store/file-to-resource-mapping.hjson",
}

## Resolvers

### from a directory

In [ ]:
config["Resolvers"] = {
    "terms": [
        {
            "resolver": "DemoResolver",
            "origin": "directory",
            "source": "../../../tests/data/demo-resolver/",
            "targets": [
                {
                    "identifier": "sex",
                    "bucket": "sex.json"
                }
            ],
            "result_resource_mapping": "../../configurations/demo-resolver/term-to-resource-mapping.hjson"
        }
    ],
    "entities": [
        {
            "resolver": "DemoResolver",
            "origin": "directory",
            "source": "../../../tests/data/demo-resolver/",
            "targets": [
                {
                    "identifier": "agents",
                    "bucket": "agents.json"
                }
            ],
            "result_resource_mapping": "../../configurations/demo-resolver/entity-to-resource-mapping.hjson"
        }
    ]
}

### from a web service

This feature is not yet implemented.

### from a store

In [ ]:
config["Resolvers"] = {
    "ontology": [
        {
            "resolver": "OntologyResolver",
            "origin": "store",
            "source": "BlueBrainNexus",
            "targets": [
                {
                    "identifier": "terms",
                    "bucket": "neurosciencegraph/datamodels"
                }
            ],
            "result_resource_mapping": "../../configurations/nexus-resolver/term-to-resource-mapping.hjson"
        }
    ],
    "agent": [
        {
            "resolver": "AgentResolver",
            "origin": "store",
            "source": "BlueBrainNexus",
            "targets": [
                {
                    "identifier": "agents",
                    "bucket": "bbp/agents"
                }
            ],
            "result_resource_mapping": "../../configurations/nexus-resolver/agent-to-resource-mapping.hjson"
        }
    ]
}

## Formatters

In [ ]:
config["Formatters"] = {
    "identifier": "https://kg.example.ch/{}/{}",
}

## Finalization

In [ ]:
import yaml

In [ ]:
with open("../../configurations/forge.yml", "w") as f:
    yaml.dump(config, f)

You are all set up!